
# Cambios de fresa en talladoras

Notebook reducido: calcula cambios de fresa por talladora, sumando piezas/scrap/ops entre cambios y duración del cambio. Genera solo `fresa_cambios.csv` en `../../data/processed`.



## Carga de datos
Se cargan las operaciones de producción forzando IDs a `string` para no perder ceros. Las rutas son relativas desde esta carpeta de notebook.


In [ ]:
import pandas as pd
from pathlib import Path

RAW_PATH = Path('../../data/raw/produccion_operaciones.csv')
# Desde notebooks/fresa/, '../../data/processed' es la carpeta de procesados del repo
CAMBIOS_CSV_PATH = Path('../../data/processed/fresa_cambios.csv')

# Forzar columnas ID a string para no perder ceros a la izquierda
string_cols = ['work_order_id', 'work_order_id_raw', 'ref_id', 'ref_id_raw', 'material_lot_id']

po = pd.read_csv(
    RAW_PATH,
    parse_dates=['ts_ini', 'ts_fin'],
    dayfirst=False,  # timestamps ya vienen ISO
    dtype={col: 'string' for col in string_cols},
)
po.head()



## Filtrado a tallado y acumulados por máquina
Ordenamos por tiempo, marcamos los cambios de fresa y calculamos acumulados de piezas/scrap/operaciones por talladora.


In [ ]:
# Filtrar tallado, ordenar y acumular
po_tall = po[po['op_id'].str.contains('TALL', na=False)].copy()
po_tall['duracion_min'] = pd.to_numeric(po_tall['duracion_min'], errors='coerce')
po_tall['piezas_ok'] = pd.to_numeric(po_tall['piezas_ok'], errors='coerce').fillna(0)
po_tall['piezas_scrap'] = pd.to_numeric(po_tall['piezas_scrap'], errors='coerce').fillna(0)
po_tall['is_change'] = po_tall['tipo_incidencia'].eq('CAMBIO DE FRESA')

po_tall = po_tall.sort_values(['machine_id', 'ts_ini']).reset_index(drop=True)

# Acumulados totales por máquina
po_tall['cum_piezas'] = po_tall.groupby('machine_id')['piezas_ok'].cumsum()
po_tall['cum_scrap'] = po_tall.groupby('machine_id')['piezas_scrap'].cumsum()
po_tall['cum_ops'] = po_tall.groupby('machine_id').cumcount() + 1
po_tall.head()



## Tabla de cambios de fresa
Para cada evento de cambio tomamos el timestamp de inicio (`ts_ini`) y restamos acumulados respecto al cambio anterior para obtener piezas/scrap/ops desde el último cambio. También calculamos la duración del cambio (`ts_fin - ts_ini`).


In [ ]:
changes = po_tall[po_tall['is_change']].copy()
changes['ts_cambio'] = changes['ts_ini']
changes['duracion_cambio_min'] = (changes['ts_fin'] - changes['ts_ini']).dt.total_seconds() / 60

changes['piezas_hasta_cambio'] = changes.groupby('machine_id')['cum_piezas'].diff().fillna(changes['cum_piezas'])
changes['scrap_hasta_cambio'] = changes.groupby('machine_id')['cum_scrap'].diff().fillna(changes['cum_scrap'])
changes['ops_hasta_cambio'] = changes.groupby('machine_id')['cum_ops'].diff().fillna(changes['cum_ops'])

cambios = changes[
    ['machine_id', 'machine_name', 'ts_cambio', 'piezas_hasta_cambio', 'scrap_hasta_cambio', 'duracion_cambio_min', 'ops_hasta_cambio']
].reset_index(drop=True)

print(cambios.head())
print('Total cambios:', len(cambios))



## Guardar CSV
Se guarda únicamente `fresa_cambios.csv` en la carpeta de procesados.


In [ ]:
CAMBIOS_CSV_PATH.parent.mkdir(parents=True, exist_ok=True)
cambios.to_csv(CAMBIOS_CSV_PATH, index=False)
CAMBIOS_CSV_PATH



## Ejemplo rápido
Primeras filas para una máquina de ejemplo (cambia `example_machine_id` si quieres inspeccionar otra).


In [ ]:
example_machine_id = cambios['machine_id'].dropna().unique()[0]
print('Ejemplo máquina:', example_machine_id)
cambios[cambios['machine_id'] == example_machine_id].head(10)
